Created on Mon Dec 01 15:05:56 2016
@author: Richard

### Required packages:
pySTATIS
numpy
mapalign
nibabel
sklearn
cluster_roi

### suggested file struture:
main/
main/cpac/filt_noglobal/rois_cc400/ > for data files
main/Affn/ > for adjacency matrices
main/Embs/ > for diffusion embedding files

### download ABIDE data:
http://preprocessed-connectomes-project.org/abide/download.html
python download_abide_preproc.py -d rois_cc400 -p cpac -s filt_noglobal -o data/ -x 'M' -gt 18 -lt 55

In [11]:
## lets start with some actual script
# import useful things
import numpy as np
import os
import nibabel as nib
from sklearn.metrics import pairwise_distances

# get a list of inputs
from os import listdir
from os.path import isfile, join

# little helper function to return the proper filelist with the full path
def listdir_nohidden(path):
    for f in os.listdir(path):
        if not f.startswith('.'):
            yield f

def listdir_fullpath(d):
    return [os.path.join(d, f) for f in listdir_nohidden(d)]
# and create a filelist
onlyfiles = listdir_fullpath("data/Outputs/cpac/filt_noglobal/rois_cc400")

In [13]:
# check to see which files contains nodes with missing information
missingarray = []
for i in onlyfiles:
# load timeseries
    filename = i
    ts_raw = np.loadtxt(filename)

# check zero columns
    missingn = np.where(~ts_raw.any(axis=0))[0]
    missingarray.append(missingn)

# select the ones that don't have missing data
ids = np.where([len(i) == 0 for i in missingarray])[0]
selected = [onlyfiles[i] for i in ids]
# could be useful to have one without pathnames later one
selected2 = [os.path.basename(onlyfiles[i]) for i in ids]
print(len(selected))

178


### run the diffusion embedding

In [14]:
# run the diffusion embedding
from mapalign import embed

for i in selected:
    # load timeseries
    print i
    filename = i
    ts = np.loadtxt(filename)
    # create correlation matrix
    dcon = np.corrcoef(ts.T)
    dcon[np.isnan(dcon)] = 0

    # Get number of nodes
    N = dcon.shape[0]

    # threshold
    perc = np.array([np.percentile(x, 90) for x in dcon])

    for ii in range(dcon.shape[0]):
        #print "Row %d" % ii
        dcon[ii, dcon[ii,:] < perc[ii]] = 0

    # If there are any left then set them to zero
    dcon[dcon < 0] = 0

    # compute the pairwise correctionlation distances
    aff = 1 - pairwise_distances(dcon, metric = 'cosine')

    # start saving
    savename = os.path.basename(filename)
    np.save("./data/Outputs/Affn/"+savename+"_cosine_affinity.npy", aff)
    # get the diffusion maps
    emb, res = embed.compute_diffusion_map(aff, alpha = 0.5)
    # Save results
    np.save("./data/Outputs/Embs/"+savename+"_embedding_dense_emb.npy", emb)
    np.save("./data/Outputs/Embs/"+savename+"_embedding_dense_res.npy", res)

    X = res['vectors']
    X = (X.T/X[:,0]).T[:,1:]    
    
    np.save("./data/Outputs/Embs/"+savename+"_embedding_dense_res_veconly.npy", X) #store vectors only

data/Outputs/cpac/filt_noglobal/rois_cc400/Caltech_0051465_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Caltech_0051474_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Caltech_0051476_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Caltech_0051477_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Caltech_0051488_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Caltech_0051492_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/CMU_a_0050649_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuven_1_0050682_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuven_1_0050683_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuven_1_0050685_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuven_1_0050686_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuven_1_0050688_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuven_1_0050689_rois_cc400.1D
data/Outputs/cpac/filt_noglobal/rois_cc400/Leuv

Run Statis to back-project the grouped embeddings

In [17]:
from pySTATIS import statis

#load vectors
names = list(xrange(392))
X = [np.load("./data/Outputs/Embs/"+ os.path.basename(filename)+"_embedding_dense_res_veconly.npy") for filename in selected2]
out = statis.statis(X, names, fname='statis_results.npy')
statis.project_back(X, out['Q'], path = "./data/Outputs/Regs/",fnames = selected2)
np.save("Mean_Vec.npy",out['F'])

Normalizing tables...
Not performing any manipulations on input matrices
Stacking matrices horizontally...
Getting masses for rows and weights for columns...
Computing Z...
Computing C...
Decomposing similarity matrix...
Performing GSVD on the horizontally concatenated matrix...
Calculating contributions of rows, columns and tables...
File already exists for subject 0
File already exists for subject 1
File already exists for subject 2
File already exists for subject 3
File already exists for subject 4
File already exists for subject 5
File already exists for subject 6
File already exists for subject 7
File already exists for subject 8
File already exists for subject 9
File already exists for subject 10
File already exists for subject 11
File already exists for subject 12
File already exists for subject 13
File already exists for subject 14
File already exists for subject 15
File already exists for subject 16
File already exists for subject 17
File already exists for subject 18
File alr

In [ ]:
# saving everything in one dump
import pickle
with open('output.pickle' ,'w') as f:
    pickle.dump([selected, out],f)

#### plotting 
plot to surface for inspection
this cell in only necessary for plotting below

In [18]:
import matplotlib.pylab as plt
import nilearn
import nilearn.plotting

import numpy as np
import nibabel as nib

def rebuild_nii(num):

    data = np.load('Mean_Vec.npy')
    a = data[:,num].copy()
    nim = nib.load('cc400_roi_atlas.nii')
    imdat=nim.get_data()
    imdat_new = imdat.copy()

    for n, i in enumerate(np.unique(imdat)):
        if i != 0:
            imdat_new[imdat == i] = a[n-1] * 100000 # scaling factor. Could also try to get float values in nifti...

    nim_out = nib.Nifti1Image(imdat_new, nim.get_affine(), nim.get_header())
    nim_out.set_data_dtype('float32')
    # to save:
    nim_out.to_filename('Gradient_'+ str(num) +'_res.nii')

    nilearn.plotting.plot_epi(nim_out)
    return(nim_out)

In [20]:
for i in range(10):
    nims = rebuild_nii(i)